In [30]:
!pip install langchain-community PyPDF2

  Using cached langchain_community-0.2.19-py3-none-any.whl.metadata (2.7 kB)
  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.22.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.2.19-py3-none-any.whl (2.3 MB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached marshmallow-3.22.0-py3-none-any.whl (49 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.1.0-py3-none-any.whl (5.0 kB)


In [13]:
import os
import json
import pandas as pd
import traceback

In [14]:
from langchain_openai import ChatOpenAI

In [26]:
from dotenv import load_dotenv
load_dotenv("/Users/ss11/mcqgen/src/.env")

True

In [24]:
KEY=os.getenv("OPENAI_API_KEY")

In [ ]:
llm=ChatOpenAI(api_key=KEY, model="gpt-3.5-turbo", temperature=0.5)

In [32]:
from langchain_community.llms import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_community.callbacks import get_openai_callback
import PyPDF2

In [49]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [50]:
TEMPLATE="""
Text={text}
Generate a quiz with {number} multiple choice questions based on the following text.
For subject, use {subject} and tone {tone}.Ensure to make {number} questions.
Make sure to format response as a JSON object response_json
## RESPONSE_JSON={response_json}

"""

In [51]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [59]:
quizChain = LLMChain(llm=llm,
                     prompt=quiz_generation_prompt,
                     output_key="quiz",
                     verbose=True)

In [60]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [61]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [62]:
generate_evaluate_chain=SequentialChain(chains=[quizChain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [43]:
file_path = "/Users/ss11/mcqgen/src/data.txt"

In [45]:
file_path

'/Users/ss11/mcqgen/src/data.txt'

In [46]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [47]:
print(TEXT)

Machine learning is a branch of Artificial Intelligence that focuses on developing models and algorithms that let computers learn from data without being explicitly programmed for every task. In simple words, ML teaches the systems to think and understand like humans by learning from the data.

Machine Learning is mainly divided into three core types: Supervised, Unsupervised and Reinforcement Learning along with two additional types, Semi-Supervised and Self-Supervised Learning.

Supervised Learning: Trains models on labeled data to predict or classify new, unseen data.
Unsupervised Learning: Finds patterns or groups in unlabeled data, like clustering or dimensionality reduction.
Reinforcement Learning: Learns through trial and error to maximize rewards, ideal for decision-making tasks.
Note: The following are not part of the original three core types of ML, but they have become increasingly important in real-world applications, especially in deep learning.

Additional Types:

Self-Su

In [63]:
json.dumps(RESPONSE_JSON, indent=4)

'{\n    "1": {\n        "mcq": "multiple choice question",\n        "options": {\n            "a": "choice here",\n            "b": "choice here",\n            "c": "choice here",\n            "d": "choice here"\n        },\n        "correct": "correct answer"\n    },\n    "2": {\n        "mcq": "multiple choice question",\n        "options": {\n            "a": "choice here",\n            "b": "choice here",\n            "c": "choice here",\n            "d": "choice here"\n        },\n        "correct": "correct answer"\n    },\n    "3": {\n        "mcq": "multiple choice question",\n        "options": {\n            "a": "choice here",\n            "b": "choice here",\n            "c": "choice here",\n            "d": "choice here"\n        },\n        "correct": "correct answer"\n    }\n}'

In [65]:
NUMBER=2
SUBJECT="Data Science"
TONE="medium"

In [66]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

/var/folders/dn/91zjh5n17zj0x4dflgnygqwc0000gp/T/ipykernel_18621/4202470493.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text=Machine learning is a branch of Artificial Intelligence that focuses on developing models and algorithms that let computers learn from data without being explicitly programmed for every task. In simple words, ML teaches the systems to think and understand like humans by learning from the data.

Machine Learning is mainly divided into three core types: Supervised, Unsupervised and Reinforcement Learning along with two additional types, Semi-Supervised and Self-Supervised Learning.

Supervised Learning: Trains models on labeled data to predict or classify new, unseen data.
Unsupervised Learning: Finds patterns or groups in unlabeled data, like clustering or dimensionality reduction.
Reinforcement Learning: Learns through trial and error to maximize rewards, ideal for decision-making tasks.
Note: The following are not part of the original three core types of ML, but they have become increasingly important in real-world applications, especially in deep learni

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [68]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0
